In [1]:
from gurobipy import *

produce = {1:[2,4], 2:[1,2,3], 3:[2,3,4]}

d = {
    (1,1):80, (1,2):85, (1,3):300, (1,4):6,
    (2,1):270, (2,2):160, (2,3):400, (2,4):7,
    (3,1):250, (3,2):130, (3,3):350, (3,4):4,
    (4,1):160, (4,2):60, (4,3):200, (4,4):3,
    (5,1):180, (5,2):40, (5,3):150, (5,4):5,
}

I = set([i for (i, k) in d])

J, M = multidict({1:3000, 2:3000, 3:3000})
K, weight = multidict({1:5, 2:2, 3:3, 4:4})

cost = {
    (1,1): 4, (1,2): 6, (1,3):9,
    (2,1): 5, (2,2): 4, (2,3):7,
    (3,1): 6, (3,2): 3, (3,3):4,
    (4,1): 8, (4,2): 5, (4,3):3,
    (5,1): 10, (5,2): 8, (5,3):4,
}

In [2]:
c = {}
for i in I:
    for j in J:
        for k in produce[j]:
            c[i, j, k] = cost[i, j] * weight[k]

In [3]:
model = Model("transportation")
x = {}
for i,j,k in c:
    x[i,j,k] = model.addVar(vtype="C")
model.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15


In [4]:
for i in I:
    for k in K:
        model.addConstr(quicksum(x[i,j,k] for j in J if (i,j,k) in x) == d[i,k])


In [5]:
for j in J:
    model.addConstr(quicksum(x[i,j,k] for (i,j2,k) in x if j2 == j) <= M[j])


In [6]:
model.setObjective(quicksum(c[i,j,k]*x[i,j,k] for (i,j,k) in x), GRB.MINIMIZE)

model.optimize()
print("Optimal value:", model.ObjVal)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23 rows, 40 columns and 80 nonzeros
Model fingerprint: 0x77b0a6c4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+03]
Presolve removed 23 rows and 40 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.3536000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.353600000e+04
Optimal value: 43536.0


In [5]:
from gurobipy import *
a = {
    (1,1):0.25, (1,2):0.15, (1,3):0.3,
    (2,1):0.3, (2,2):0.3, (2,3):0.1,
    (3,1):0.15, (3,2):0.65, (3,3):0.05,
    (4,1):0.1, (4,2):0.05, (4,3):0.85,
}
I, p = multidict({1:5, 2:6, 3:8, 4:20})
K, LB, UB = multidict({1:[0.1,0.2], 2:[0.0,0.35], 3:[0.45,1.0]})

{1: 0.1, 2: 0.0, 3: 0.45}


In [20]:
model = Model("mix")
x = {}
for i in I:
    x[i]=model.addVar()
model.update()
model.addConstr(quicksum(x[i] for i in I) == 1)
for k in K:
    model.addConstr(quicksum(a[i, k]*x[i] for i in I) <= UB[k])
    model.addConstr(quicksum(a[i, k]*x[i] for i in I) >= LB[k])
model.setObjective(quicksum(p[i]*x[i] for i in I), GRB.MINIMIZE)
model.optimize()
for i in I:
    print(i, x[i].X)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 4 columns and 28 nonzeros
Model fingerprint: 0xd2a78047
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [5e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+00]
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0372443e+00   7.173123e-01   0.000000e+00      0s
       2    9.6216216e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.621621622e+00
1 0.6486486486486487
2 0.0
3 0.05405405405405392
4 0.29729729729729737
